In [2]:
from socket import *
import base64
import time
import ssl

msg = "\r\n I love computer networks!"
endmsg = "\r\n.\r\n"
mailserver = ("smtp.gmail.com", 587) #Fill in start #Fill in end

#Create the socket and connect with the mail server
clientSocket = socket(AF_INET, SOCK_STREAM)
clientSocket.connect(mailserver)
recv = clientSocket.recv(1024)
recv = recv.decode()
print("Message after connection request:" + recv)
if recv[:3] != '220':
    print('220 reply not received from server.')
#Send hello message to the mail server
heloCommand = 'EHLO Alice\r\n'
clientSocket.send(heloCommand.encode())
recv1 = clientSocket.recv(1024)
recv1 = recv1.decode()
print("Message after EHLO command:" + recv1)
if recv1[:3] != '250':
    print('250 reply not received from server.')

#Use SSL for authentication with the server. Indicate that you would use an SSL connection
command = "STARTTLS\r\n"
clientSocket.send(command.encode())
recv5 = clientSocket.recv(1024)
recv5 = recv5.decode()
print("After STARTTLS FROM command: "+recv5)

#Create an SSL socket on existing clientSocket and perform handshake again
sslSocket = ssl.wrap_socket(clientSocket, ssl_version=ssl.PROTOCOL_SSLv23)
heloCommand = 'EHLO Alice\r\n'
sslSocket.send(heloCommand.encode())
recv6 = sslSocket.recv(1024)
recv6 = recv6.decode()
print("After Ehlo FROM command: "+recv6)
#sslSocket.send("AUTH PLAIN ".encode()+"\r\n".encode())
#username = "xxxxxxxxxxxxxxx".encode()+"\r\n".encode()
#sslSocket.send(base64.b64encode(username))

#password = "xxxxxxxxxxxxxxxx".encode()+"\r\n".encode()
#sslSocket.send(base64.b64encode(password))

#Prepare the authentication message with username and password
username = "anshul.gautam@gmail.com"
password = "Ansh6701855!"

base64_str = ("\x00"+username+"\x00"+password).encode()
base64_str = base64.b64encode(base64_str)
authMsg = "AUTH PLAIN ".encode()+base64_str+"\r\n".encode()
sslSocket.send(authMsg)
recv_auth = sslSocket.recv(1024)
print(recv_auth.decode())

#Prepare the message for sending to the recepient
mailFrom = "MAIL FROM:<anshul.gautam@gmail.com>\r\n"
sslSocket.send(mailFrom.encode())
recv2 = sslSocket.recv(1024)
recv2 = recv2.decode()
print("After MAIL FROM command: "+recv2)
rcptTo = "RCPT TO:<anshul_gautam@hotmail.com>\r\n"
sslSocket.send(rcptTo.encode())
recv3 = sslSocket.recv(1024)
recv3 = recv3.decode()
print("After RCPT TO command: "+recv3)
data = "DATA\r\n"
sslSocket.send(data.encode())
recv4 = sslSocket.recv(1024)
recv4 = recv4.decode()
print("After DATA command: "+recv4)
subject = "Subject: testing my client\r\n\r\n" 
sslSocket.send(subject.encode())
date = time.strftime("%a, %d %b %Y %H:%M:%S +0000", time.gmtime())
date = date + "\r\n\r\n"
sslSocket.send(date.encode())
sslSocket.send(msg.encode())
sslSocket.send(endmsg.encode())
recv_msg = sslSocket.recv(1024)
print("Response after sending message body and period:"+recv_msg.decode())

#Close the connection
quit = "QUIT\r\n"
sslSocket.send(quit.encode())
recv5 = sslSocket.recv(1024)
print(recv5.decode())
sslSocket.close()
clientSocket.close()

Message after connection request:220 smtp.gmail.com ESMTP k76sm1612162ita.4 - gsmtp

Message after EHLO command:250-smtp.gmail.com at your service, [65.28.255.7]
250-SIZE 35882577
250-8BITMIME
250-STARTTLS
250-ENHANCEDSTATUSCODES
250-PIPELINING
250-CHUNKING
250 SMTPUTF8

After STARTTLS FROM command: 220 2.0.0 Ready to start TLS

After Ehlo FROM command: 250-smtp.gmail.com at your service, [65.28.255.7]
250-SIZE 35882577
250-8BITMIME
250-AUTH LOGIN PLAIN XOAUTH2 PLAIN-CLIENTTOKEN OAUTHBEARER XOAUTH
250-ENHANCEDSTATUSCODES
250-PIPELINING
250-CHUNKING
250 SMTPUTF8

235 2.7.0 Accepted

After MAIL FROM command: 250 2.1.0 OK k76sm1612162ita.4 - gsmtp

After RCPT TO command: 250 2.1.5 OK k76sm1612162ita.4 - gsmtp

After DATA command: 354  Go ahead k76sm1612162ita.4 - gsmtp

Response after sending message body and period:250 2.0.0 OK 1507340482 k76sm1612162ita.4 - gsmtp

221 2.0.0 closing connection k76sm1612162ita.4 - gsmtp

